In [ ]:
import numpy as np 
import pandas as pd
import matplotlib as mpl 
import matplotlib.pyplot as plt
import seaborn as sns

mvz = pd.read_excel("Agg_Moovizy_V3.xlsx", index_col = 0)

modif_1 = mvz[['Date', 'Interval', 'Req', 'Req_2H', 'Req_1D', 'Req_DP',
       'TopSpeed (m/s)', 'Num_Transfer', 'Weekend', 'Day',
       'Total Ticket Sales ', 'Regular User ', 'Irregular Users']]

dt_mvz = modif_1.rename(columns={'TopSpeed (m/s)':'TopSpeed',
                        'Total Ticket Sales ':'Ticket_Sales',
                        'Regular User ':'Regular_Sales',
                        'Irregular Users':'Irregular_Sales'})

In [ ]:
idx = dt_mvz.TopSpeed != np.inf
noinf = dt_mvz[idx]
noinf_mean = np.mean(noinf.TopSpeed)

dt_mvz.replace([np.inf, -np.inf], noinf_mean, inplace=True)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


model = smf.ols('Ticket_Sales ~ Req_2H + Req_1D + Req_DP + TopSpeed + Num_Transfer + C(Weekend):C(Interval)',
                  data=dt_mvz)
results = model.fit(cov_type='HC3') # heteroscedasticity robust covariance
print(results.summary())

In [ ]:
results.params[-5:] # Type = pd.Series

In [ ]:
results.rsquared

In [ ]:
sm.graphics.plot_partregress(endog='Ticket_Sales', exog_i='Req_2H',
                             exog_others=['Req_1D', 'Req_DP', 'TopSpeed', 'Num_Transfer', 'C(Weekend):C(Interval)'],
                             alpha=.4, data=dt_mvz, obs_labels=False)

sm.graphics.plot_partregress(endog='Ticket_Sales', exog_i='Req_1D',
                             exog_others=['Req_2H', 'Req_DP', 'TopSpeed', 'Num_Transfer', 'C(Weekend):C(Interval)'],
                             alpha=.4, data=dt_mvz, obs_labels=False)

sm.graphics.plot_partregress(endog='Ticket_Sales', exog_i='Req_DP',
                             exog_others=['Req_2H', 'Req_1D', 'TopSpeed', 'Num_Transfer', 'C(Weekend):C(Interval)'],
                             alpha=.4, data=dt_mvz, obs_labels=False)